## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append("..")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ayagoz,KZ,47.9714,80.4392,45.64,48,0,8.70,clear sky
1,1,Caravelas,BR,-17.7125,-39.2481,78.49,73,1,10.25,clear sky
2,2,Chuy,UY,-33.6971,-53.4616,53.51,95,100,12.24,overcast clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,64.51,73,76,31.74,broken clouds
4,4,Agadez,NE,19.7500,10.2500,94.35,13,95,5.57,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))


What is the minimum temperature you would like for your vacation?75
What is the maximum temperature you would like for your vacation?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp)
                           & (city_data_df["Max Temp"] <= max_temp)]
pref_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Caravelas,BR,-17.7125,-39.2481,78.49,73,1,10.25,clear sky
13,13,Bardiyah,LY,31.7561,25.0865,76.98,66,0,13.91,clear sky
16,16,Belmonte,BR,-15.8631,-38.8828,82.90,67,0,10.63,clear sky
20,20,Dirba,IN,30.0667,75.9833,80.01,78,46,6.42,scattered clouds
23,23,Sur,OM,22.5667,59.5289,84.79,69,100,11.79,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
pref_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pref_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Caravelas,BR,78.49,clear sky,-17.7125,-39.2481,
13,Bardiyah,LY,76.98,clear sky,31.7561,25.0865,
16,Belmonte,BR,82.90,clear sky,-15.8631,-38.8828,
20,Dirba,IN,80.01,scattered clouds,30.0667,75.9833,
23,Sur,OM,84.79,overcast clouds,22.5667,59.5289,
27,Nicoya,CR,85.55,light rain,10.1483,-85.4520,
28,Georgetown,MY,78.89,few clouds,5.4112,100.3354,
30,Salinopolis,BR,83.10,overcast clouds,-0.6136,-47.3561,
34,Manono,CD,85.23,broken clouds,-7.3000,27.4167,
38,Warri,NG,78.37,scattered clouds,5.5167,5.7500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found . . . Skipping")
    
        

Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping
Hotel not found . . . Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
#Check what the dataframe looks like so far
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Caravelas,BR,78.49,clear sky,-17.7125,-39.2481,Pousada dos Navegantes
13,Bardiyah,LY,76.98,clear sky,31.7561,25.0865,بورتو بردية
16,Belmonte,BR,82.90,clear sky,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
20,Dirba,IN,80.01,scattered clouds,30.0667,75.9833,Victoria Resort
23,Sur,OM,84.79,overcast clouds,22.5667,59.5289,Sur Plaza Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))